In [1]:
from __future__ import print_function
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras import callbacks
import PIL
import h5py
from PIL import Image

C:\Users\tra-anh.nguyen\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_data_path = './data/train'
validation_data_path = './data/validation'
test_data_path = './data/test'

In [3]:
batch_size = 4
epochs = 4

In [4]:
# Build the model structure 
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64,64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
classifier.add(Dense(64))
classifier.add(Activation('relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(1))
classifier.add(Activation('sigmoid'))
classifier.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Path for train set
training_set = train_datagen.flow_from_directory(train_data_path,
target_size = (64, 64),
batch_size = batch_size,
class_mode = 'binary')

# Path for test set
test_set = test_datagen.flow_from_directory(test_data_path,
target_size = (64, 64),
batch_size = batch_size,
class_mode = 'binary')

#Path for validation set
validation_set = test_set = test_datagen.flow_from_directory(test_data_path,
target_size = (64, 64),
batch_size = batch_size,
class_mode = 'binary')

Found 321 images belonging to 2 classes.
Found 37 images belonging to 2 classes.
Found 37 images belonging to 2 classes.


In [6]:
classifier.fit_generator(training_set,
steps_per_epoch = 1000,
epochs = epochs,
validation_data = validation_set,
validation_steps = 100)

Epoch 1/4
1000/1000 [==============================] - 651s 651ms/step - loss: 3.2331 - acc: 0.7963 - val_loss: 3.8779 - val_acc: 0.7568
Epoch 2/4
1000/1000 [==============================] - 586s 586ms/step - loss: 3.2399 - acc: 0.7967 - val_loss: 3.8779 - val_acc: 0.7568
Epoch 3/4
1000/1000 [==============================] - 623s 623ms/step - loss: 3.2363 - acc: 0.7970 - val_loss: 3.8779 - val_acc: 0.7568
Epoch 4/4
1000/1000 [==============================] - 608s 608ms/step - loss: 3.2403 - acc: 0.7967 - val_loss: 3.8779 - val_acc: 0.7568


In [16]:
saved_model = classifier
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1968192   
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
__________

In [17]:
# how well did it do ?

#score = classifier.evaluate(test_set, verbose = 0)
#print('Test loss: ', score[0])
#print('Test accuracy: ', score[1])

In [9]:

# Save the model
# Serialize the model to JSON

model_json = classifier.to_json()
with open("classifier.json", "w") as json_file:
	json_file.write(model_json)

In [10]:

# Save model and Serialize weights to HDF5

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
classifier.save('./models/model.h5')
classifier.save_weights('./models/weights.h5')	
print("Saved model to disk")


Saved model to disk
